#**Import libraries**

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

import statsmodels.stats.multicomp as smm
import scipy.stats as stats
from statsmodels.formula.api import ols
from statsmodels.stats import weightstats
from statsmodels.stats.weightstats import ztest

#**Download dataset**

In [ ]:
#https://drive.google.com/file/d/113Y-jwWvkUHShZg8azynF94k8OpC0ZQ0/view?usp=sharing
!gdown --id 113Y-jwWvkUHShZg8azynF94k8OpC0ZQ0

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=113Y-jwWvkUHShZg8azynF94k8OpC0ZQ0
To: /content/weatherAUS_cleaned.csv
100% 16.0M/16.0M [00:00<00:00, 73.1MB/s]


# Review dataset

In [ ]:
df = pd.read_csv('/content/weatherAUS_cleaned.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131941 entries, 0 to 131940
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Date             131941 non-null  object 
 1   Location         131941 non-null  object 
 2   MinTemp          131941 non-null  float64
 3   MaxTemp          131941 non-null  float64
 4   Rainfall         131941 non-null  float64
 5   Sunshine         131941 non-null  float64
 6   WindGustDir      131941 non-null  object 
 7   WindGustSpeed    131941 non-null  float64
 8   WindDir9am       131941 non-null  object 
 9   WindDir3pm       131941 non-null  object 
 10  WindSpeed9am     131941 non-null  float64
 11  WindSpeed3pm     131941 non-null  float64
 12  Humidity9am      131941 non-null  float64
 13  Humidity3pm      131941 non-null  float64
 14  Pressure9am      131941 non-null  float64
 15  Pressure3pm      131941 non-null  float64
 16  Temp9am          131941 non-null  floa

In [ ]:
df.head(2)

,Date,Location,MinTemp,MaxTemp,Rainfall,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,is_RainToday,is_RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,7.624853,W,44.0,W,WNW,...,71.0,22.0,1007.7,1007.1,16.9,21.8,No,No,0.0,0.0
1,2008-12-02,Albury,7.4,25.1,0.0,7.624853,WNW,44.0,NNW,WSW,...,44.0,25.0,1010.6,1007.8,17.2,24.3,No,No,0.0,0.0


In [ ]:
categorical_cols = df.select_dtypes(exclude='number').columns.tolist()
categorical_cols = categorical_cols[:-1]
print('Các cột categorical:')
print(categorical_cols)
print('----------------------')
numerical_cols = df.select_dtypes(exclude='object').columns.tolist()
print('Các cột numerical:')
print(numerical_cols)

Các cột categorical:
['Date', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday']
----------------------
Các cột numerical:
['MinTemp', 'MaxTemp', 'Rainfall', 'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm', 'is_RainToday', 'is_RainTomorrow']


In [ ]:
df_cat = df.select_dtypes(include='object')

In [ ]:
col_name = df_cat.columns.tolist()
for i in col_name:
  print(f'{i}:     {df_cat[i].unique()}')

Date:     ['2008-12-01' '2008-12-02' '2008-12-03' ... '2008-11-23' '2008-12-13'
 '2008-11-22']
Location:     ['Albury' 'BadgerysCreek' 'Cobar' 'CoffsHarbour' 'Moree' 'Newcastle'
 'NorahHead' 'NorfolkIsland' 'Penrith' 'Richmond' 'Sydney' 'SydneyAirport'
 'WaggaWagga' 'Williamtown' 'Wollongong' 'Canberra' 'Tuggeranong'
 'MountGinini' 'Ballarat' 'Bendigo' 'Sale' 'MelbourneAirport' 'Melbourne'
 'Mildura' 'Nhil' 'Portland' 'Watsonia' 'Dartmoor' 'Brisbane' 'Cairns'
 'GoldCoast' 'Townsville' 'Adelaide' 'MountGambier' 'Nuriootpa' 'Woomera'
 'Albany' 'Witchcliffe' 'PearceRAAF' 'PerthAirport' 'Perth' 'SalmonGums'
 'Walpole' 'Hobart' 'Launceston' 'AliceSprings' 'Darwin' 'Katherine'
 'Uluru']
WindGustDir:     ['W' 'WNW' 'WSW' 'NE' 'N' 'NNE' 'SW' 'ENE' 'SSE' 'S' 'NNW' 'NW' 'SE' 'E'
 'SSW' 'ESE']
WindDir9am:     ['W' 'NNW' 'SE' 'ENE' 'SW' 'SSE' 'S' 'NE' 'N' 'SSW' 'WSW' 'ESE' 'E' 'NW'
 'WNW' 'NNE']
WindDir3pm:     ['WNW' 'WSW' 'E' 'NW' 'W' 'SSE' 'ESE' 'ENE' 'SSW' 'SW' 'NNW' 'SE' 'N' 'S'
 'NNE' 'NE']


# Hàm tra trị số z theo phân phối chuẩn

In [ ]:
## Hàm tính xác suất theo phân phối Z
def zDistribution(side, lower, upper):
    side = side.lower()
    if (side == 'left'):
        p = stats.norm.cdf(lower)
    elif (side == 'right'):
        p = 1 - stats.norm.cdf(upper)
    else:
        p = stats.norm.cdf(upper) - stats.norm.cdf(lower)
    return (p)

## Hàm hiển thị xác suất
def displayZ(side, lower, upper, p):
    side = side.lower()
    lower = str(lower)
    upper = str(upper)
    if (side == 'left'):
        s = 'P(Z < ' + lower + ') = '
    elif (side == 'right'):
        s = 'P(' + upper + ' < Z) = '
    else:
        s = 'P(' + lower + ' < Z < ' + upper + ') = '
    return (s + str('%.4f' %p))

# **Kiểm định z_test**

## 1.1. Kiểm định giả thuyết: Nhiệt độ vào ban ngày sẽ ấm hơn ban đêm

In [ ]:
tem9am = df['Temp9am']
tem3pm = df['Temp3pm']

In [ ]:
print(f'Số quan sát của nhóm nhiệt độ lúc 9am: {tem9am.shape[0]}')
print(f'Số quan sát của nhóm nhiệt độ lúc 3pm: {tem3pm.shape[0]}')

Số quan sát của nhóm nhiệt độ lúc 9am: 131941
Số quan sát của nhóm nhiệt độ lúc 3pm: 131941


In [ ]:
print(f'Giá trị nhiệt độ trung bình lúc 9am: {tem9am.mean():.4f}')
print(f'Giá trị nhiệt độ trung bình lúc 3pm: {tem3pm.mean():.4f}')

Giá trị nhiệt độ trung bình lúc 9am: 16.9122
Giá trị nhiệt độ trung bình lúc 3pm: 21.8008


In [ ]:
alpha = 0.05 # Mức ý nghĩa 5%
z3, p3 = ztest(tem9am , value = tem3pm.mean(), alternative = 'larger')
print(f'Giá trị z_score: ~ {z3:.4f} \nGiá trị p_value: ~ {p3}' )

Giá trị z_score: ~ -279.3686 
Giá trị p_value: ~ 1.0


In [ ]:
if (p3 < alpha):
  print(f'p_value: {p3} =< alpha: {alpha} => Có cơ sở để Bác bỏ giá trị H0 \n=> Nhiệt độ vào buổi sáng cao hơn buổi tối')
else:
  print(f'p_value: {p3} > alpha: {alpha} => Không có cơ sở để Bác bỏ giá trị H0 \n=> Nhiệt độ vào buổi tối cao hơn buổi sáng')

p_value: 1.0 > alpha: 0.05 => Không có cơ sở để Bác bỏ giá trị H0 
=> Nhiệt độ vào buổi tối cao hơn buổi sáng


## 1.2. Kiểm định giả thuyết: Ngày hôm nay có mưa không ảnh hưởng đến nhiệt độ tối thiểu
H0: μ(`MinTemp`)[`'is_RainToday' == 1`] = μ(`MinTemp`)[`'is_RainToday' == 0`]

H1: μ(`MinTemp`)[`'is_RainToday' == 1`] != μ(`MinTemp`)[`'is_RainToday' == 0`]

In [ ]:
#kiểm tra tính đối xứng
print('Kiểm tra cân bằng')
df['is_RainToday'].value_counts()

Kiểm tra cân bằng


0.0    105497
1.0     26444
Name: is_RainToday, dtype: int64

In [ ]:
temp_rain = df[(df['is_RainToday'] == 1)]['MinTemp']
temp_nrain = df[(df['is_RainToday'] == 0)]['MinTemp']

temp_rain_array = np.array(temp_rain)
temp_nrain_array = np.array(temp_nrain)

variance_rain = (temp_rain_array).var(ddof = 1)
variance_nrain = (temp_nrain_array).var(ddof = 1)

print(f'Phương sai nhiệt độ tối thiểu của có mưa hôm nay: {variance_rain:.2f}')
print(f'Phương sai nhiệt độ tối thiểu của không có mưa hôm nay: {variance_nrain:.2f}')

Phương sai nhiệt độ tối thiểu của có mưa hôm nay: 33.83
Phương sai nhiệt độ tối thiểu của không có mưa hôm nay: 41.16


In [ ]:
mean_rain = (temp_rain_array).mean()
mean_nrain = (temp_nrain_array).mean()
print(f'Trung bình nhiệt độ tối thiểu của có mưa hôm nay: {mean_rain:.2f}')
print(f'Trung bình nhiệt độ tối thiểu của không có mưa hôm nay: {mean_nrain:.2f}')

Trung bình nhiệt độ tối thiểu của có mưa hôm nay: 12.66
Trung bình nhiệt độ tối thiểu của không có mưa hôm nay: 11.90


In [ ]:
alpha = 0.05
confidence_level = 1 - alpha

col1 = weightstats.DescrStatsW(temp_rain)
col2 = weightstats.DescrStatsW(temp_nrain)
cm_obj = weightstats.CompareMeans(col1, col2)
z, p = cm_obj.ztest_ind(usevar='unequal')
if (p < alpha):
    print(f'Trị số p = {p:.4f} < {alpha} nên Có cơ sở để Bác bỏ H0.\n=> μ[rain] != μ[nrain]')
else:
    print(f'Trị số p = {p:.4f} >= {alpha} nên Có cơ sở để Chấp nhận H0.\n=> μ[rain] == μ[nrain]')

Trị số p = 0.0000 < 0.05 nên Có cơ sở để Bác bỏ H0.
=> μ[rain] != μ[nrain]


In [ ]:
print(f'Giá trị z: {z:.4f}')

Giá trị z: 18.6695


## 1.3. Kiểm định giả thuyết: "Tốc độ gió có khả năng dự báo mưa trong tương lai gần"

In [ ]:
# Kiểm định giả thuyết
# H0: muy[No] = muy[Yes]
# H1: muy[No] != muy[Yes]

In [ ]:
rain = df.loc[df['Rainfall'] == 0]['WindGustSpeed']
no_rain = df.loc[df['Rainfall'] != 0]['WindGustSpeed']
print(f'Số quan sát của nhóm có mưa: {rain.shape[0]}')
print(f'Số quan sát của nhóm không có mưa: {no_rain.shape[0]}')

Số quan sát của nhóm có mưa: 86236
Số quan sát của nhóm không có mưa: 45705


In [ ]:
#Tính trung bình 2 nhóm
print(f'Giá trị trung bình tốc độ gió của nhóm có mưa: {rain.mean():.4f}')
print(f'Giá trị trung bình tốc độ gió nhóm không có mưa: {no_rain.mean():.4f}')

Giá trị trung bình tốc độ gió của nhóm có mưa: 38.1180
Giá trị trung bình tốc độ gió nhóm không có mưa: 40.0699


In [ ]:
alpha = 0.05 # Mức ý nghĩa 5%
z1, p1 = ztest(rain , value = no_rain.mean(), alternative='two-sided')
print(f'Giá trị z_score ~ {z1:.4f} \nGiá trị p_value ~ {p1}' )

Giá trị z_score ~ -51.2867 
Giá trị p_value ~ 0.0


In [ ]:
if (p1 < alpha):
  print(f'p_value: {p1} =< alpha: {alpha} => Có cơ sở để Bác bỏ giá trị H0 \n=> Tốc độ gió có giả năng dự đoán mưa trong tương lai gần')
else:
  print(f'p_value: {p1} > alpha: {alpha} => Không có sở sở để Bác bỏ giá trị H0 \n=> Tốc độ gió không có giả năng dự đoán mưa trong tương lai gần')

p_value: 0.0 =< alpha: 0.05 => Có cơ sở để Bác bỏ giá trị H0 
=> Tốc độ gió có giả năng dự đoán mưa trong tương lai gần


In [ ]:
# Kiểm định giả thuyết
# H0: muy[No] = muy[Yes]
# H1: Có ít nhất một muy != phần còn lại

## 1.4. Kiểm định giả thuyết: "Áp suất vào ban đêm có khả năng dự báo mưa trong tương lai gần"

In [ ]:
# Kiểm định giả thuyết
# H0: muy[No] = muy[Yes]
# H1: muy[No] != muy[Yes]

In [ ]:
rain = df.loc[(df['RainTomorrow'] == 'Yes')]['Pressure3pm']
no_rain = df.loc[(df['RainTomorrow'] == 'No')]['Pressure3pm']
print(f'Số quan sát của nhóm có mưa: {rain.shape[0]}')
print(f'Số quan sát của nhóm không có mưa: {no_rain.shape[0]}')

Số quan sát của nhóm có mưa: 26791
Số quan sát của nhóm không có mưa: 105150


In [ ]:
# Tính trung bình 2 nhóm
print(f'Giá trị trung bình áp suất vào ban đêm của nhóm có mưa: {rain.mean():.4f}')
print(f'Giá trị trung bình áp suất vào ban đêm nhóm không có mưa: {no_rain.mean():.4f}')


Giá trị trung bình áp suất vào ban đêm của nhóm có mưa: 1013.2946
Giá trị trung bình áp suất vào ban đêm nhóm không có mưa: 1016.2044


In [ ]:
alpha = 0.05 # Mức ý nghĩa 5%
z1, p1 = ztest(rain , value = no_rain.mean())
print(f'Giá trị z_score: {z1:.4f} \nGiá trị p_value: {p1}' )

Giá trị z_score: -73.8934 
Giá trị p_value: 0.0


In [ ]:
if (p1 < alpha):
  print(f'p_value: {p1} =< alpha: {alpha} =>  Có cơ sở để Bác bỏ giá trị H0 \n=> Áp suất vào ban đêm có giả năng dự đoán mưa trong tương lai gần')
else:
  print(f'p_value: {p1} > alpha: {alpha} => Không có sở sở để Bác bỏ giá trị H0 \n=> Áp suất vào ban đêm không có giả năng dự đoán mưa trong tương lai gần')

p_value: 0.0 =< alpha: 0.05 =>  Có cơ sở để Bác bỏ giá trị H0 
=> Áp suất vào ban đêm có giả năng dự đoán mưa trong tương lai gần


In [ ]:
# Kiểm định giả thuyết
# H0: muy[No] = muy[Yes]
# H1: Có ít nhất một muy != phần còn lại

## 1.5.Kiểm định giả thuyết: Tốc độ gió vào buổi tối sẽ mạnh hơn vào buổi sáng

In [ ]:
wind9am = df['WindSpeed9am']
wind3pm = df['WindSpeed3pm']

In [ ]:
print(f'Số quan sát của nhóm tốc độ gió lúc 9am: {wind9am.shape[0]}')
print(f'Số quan sát của nhóm tốc độ gió lúc 3pm: {wind3pm.shape[0]}')

Số quan sát của nhóm tốc độ gió lúc 9am: 131941
Số quan sát của nhóm tốc độ gió lúc 3pm: 131941


In [ ]:
#Tính trung bình 2 nhóm
print(f'Giá trị trung bình tốc độ gió lúc 9am: {wind9am.mean():.4f}')
print(f'Giá trị trung bình tốc độ gió lúc 3pm: {wind3pm.mean():.4f}')

Giá trị trung bình tốc độ gió lúc 9am: 13.4474
Giá trị trung bình tốc độ gió lúc 3pm: 18.1476


In [ ]:
alpha = 0.05 # Mức ý nghĩa 5%
z2, p2 = ztest(wind3pm , value = wind9am.mean(), alternative = 'larger')
print(f'Giá trị z_score: ~ {z2:.4f} \nGiá trị p_value: ~ {p2}' )

Giá trị z_score: ~ 210.3321 
Giá trị p_value: ~ 0.0


In [ ]:
if (p2 < alpha):
  print(f'p_value: {p2} =< alpha: {alpha} => Có cơ sở để Bác bỏ giá trị H0 \n=> Tốc độ gió vào buổi tối mạnh hơn buổi sáng')
else:
  print(f'p_value: {p2} > alpha: {alpha} => Không có sở sở để Bác bỏ giá trị H0 \n=> Tốc độ gió vào buổi tối yếu hơn buổi sáng')

p_value: 0.0 =< alpha: 0.05 => Có cơ sở để Bác bỏ giá trị H0 
=> Tốc độ gió vào buổi tối mạnh hơn buổi sáng


# **Kiểm định Chi-Squared**

In [ ]:
##----------------------------------------------------
## Các giả thuyết kiểm định
##    H0: Mùa và Có mưa là ĐỘC LẬP
##    Ha: Mùa và Có mưa là PHỤ THUỘC lẫn nhau
##----------------------------------------------------
alpha            = .05
confidence_level = (1 - alpha)

In [ ]:
direction_mapping = {'W': 'West', 'WNW': 'West', 'NE': 'North', 'N':'North',
                     'SSE':'South', 'S':'South', 'SW':'South', 'NNW':'North',
                     'SE':'South', 'WSW':'West', 'NNE':'North','E':'East',
                     'NW':'North', 'SSW':'South', 'ESE':'East', 'ENE':'East'}
df['WindGustDir'] = df['WindGustDir'].map(direction_mapping)
windD_counts = df['WindGustDir'].value_counts()
windD_per = windD_counts/df.shape[0]

In [ ]:
chi2_test = pd.crosstab(df['WindGustDir'], df['RainToday'])
print(chi2_test)

RainToday       No   Yes
WindGustDir             
East         20066  3190
North        29113  5341
South        32242  9661
West         24076  8252


In [ ]:
chi2_test = pd.crosstab(df['WindGustDir'], df['RainToday'])
chi2_test.reset_index(inplace = True)
print(chi2_test)

RainToday WindGustDir     No   Yes
0                East  20066  3190
1               North  29113  5341
2               South  32242  9661
3                West  24076  8252


In [ ]:
chi2_test = chi2_test.drop(columns = 'WindGustDir')
stat, p, dof, expected = stats.chi2_contingency(chi2_test)
print(f'Giá trị p_value ~ {p} \nTrị số thống kê ~ {stat:.4f}')

Giá trị p_value ~ 0.0 
Trị số thống kê ~ 1867.8903


In [ ]:
if (p < alpha):
    print(f'Trị số p = {p:} < {alpha:.4f} nên CÓ cơ sở để Bác bỏ H0 \n==> (WindGustDir, RainToday) PHỤ THUỘC')
else:
    print(f'Trị số p = {p:} >= {alpha:.4f} nến KHÔNG có cơ sở để bác bỏ H0 \n==> (WindGustDir, giới tính) ĐỘC LẬP')

Trị số p = 0.0 < 0.0500 nên CÓ cơ sở để Bác bỏ H0 
==> (WindGustDir, RainToday) PHỤ THUỘC
